<a href="https://colab.research.google.com/github/vsolodkyi/NeuralNetworks_SkillBox/blob/main/module_14/14_homework_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import tensorflow as tf
print(tf.__version__)

import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

2.8.2


In [189]:
#dataset - https://drive.google.com/file/d/1CVRGIvWpymsqopO-22VD1CkatAE5OLJH/view?usp=sharing

In [7]:
train_all = pd.read_csv('train_data.txt', sep=':::', header=None, names=['ID', 'TITLE', 'GENRE', 'DESCRIPTION'])
test = pd.read_csv('test_data.txt', sep=':::', header=None, names=['ID', 'TITLE', 'DESCRIPTION'])
test_solution = pd.read_csv('test_data_solution.txt', sep=':::', header=None, names=['ID', 'TITLE', 'GENRE', 'DESCRIPTION'])
print(train_all.shape)
print(test.shape)
print(test_solution.shape)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


(54214, 4)
(54200, 3)
(54200, 4)


In [9]:
train_all.GENRE.unique()

array([' drama ', ' thriller ', ' adult ', ' documentary ', ' comedy ',
       ' crime ', ' reality-tv ', ' horror ', ' sport ', ' animation ',
       ' action ', ' fantasy ', ' short ', ' sci-fi ', ' music ',
       ' adventure ', ' talk-show ', ' western ', ' family ', ' mystery ',
       ' history ', ' news ', ' biography ', ' romance ', ' game-show ',
       ' musical ', ' war '], dtype=object)

**Text Pre-processing**

In [15]:
train_all.DESCRIPTION[0]

' Listening in to a conversation between his doctor and parents, 10-year-old Oscar learns what nobody has the courage to tell him. He only has a few weeks to live. Furious, he refuses to speak to anyone except straight-talking Rose, the lady in pink he meets on the hospital stairs. As Christmas approaches, Rose uses her fantastical experiences as a professional wrestler, her imagination, wit and charm to allow Oscar to live life and love to the full, in the company of his friends Pop Corn, Einstein, Bacon and childhood sweetheart Peggy Blue.'

**Our text preprocessing will include the following steps:**

Convert all text to lower case.

Replace REPLACE_BY_SPACE_RE symbols by space in text.

Remove symbols that are in BAD_SYMBOLS_RE from text.

Remove “x” in text.

Remove stop words.

Remove digits in text.

In [57]:
#import re
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize

import string

# дополнительный словарь со знаками пунктуации
nltk.download('punkt', download_dir='.')

[nltk_data] Downloading package punkt to ....
[nltk_data]   Package punkt is already up-to-date!


True

In [58]:

train_all = train_all.reset_index(drop=True)
#REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]-')
#BAD_SYMBOLS_RE = re.compile('[^0-9#a-z# +_]')


In [59]:
stop_words = [
    'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd",
    'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers',
    'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which',
    'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been',
    'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if',
    'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between',
    'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out',
    'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why',
    'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not',
    'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'shold',
    "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't",
    'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't",
    'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't",
    'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"
]

In [60]:
def tokenize_text(raw_text: str):
    """Функция для токенизации текста
    
    :param raw_text: исходная текстовая строка
    """
    #raw_text = REPLACE_BY_SPACE_RE.sub(' ', raw_text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    #raw_text = BAD_SYMBOLS_RE.sub('', raw_text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
   
    tokenized_str = nltk.word_tokenize(raw_text)
    tokens = [i.lower() for i in tokenized_str if ( i not in string.punctuation )]
    filtered_tokens = [i for i in tokens if ( i not in stop_words )]

    return filtered_tokens



In [61]:
# применяем функцию в датафрейму с помощью метода .apply()
tokenized_desc = train_all['DESCRIPTION'].apply(tokenize_text)

# добавляем новую колонку в исходный датафрейм
train_all = train_all.assign(
    descr_new=tokenized_desc
)

train_all.descr_new.head()

0    [listening, conversation, doctor, parents, 10-...
1    [brother, sister, past, incestuous, relationsh...
2    [bus, empties, students, field, trip, museum, ...
3    [help, unemployed, father, make, ends, meet, e...
4    [film, 's, title, refers, un-recovered, bodies...
Name: descr_new, dtype: object

In [63]:
train_all.DESCRIPTION[0]

' Listening in to a conversation between his doctor and parents, 10-year-old Oscar learns what nobody has the courage to tell him. He only has a few weeks to live. Furious, he refuses to speak to anyone except straight-talking Rose, the lady in pink he meets on the hospital stairs. As Christmas approaches, Rose uses her fantastical experiences as a professional wrestler, her imagination, wit and charm to allow Oscar to live life and love to the full, in the company of his friends Pop Corn, Einstein, Bacon and childhood sweetheart Peggy Blue.'

In [69]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [65]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 20000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 256
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(train_all['descr_new'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 147789 unique tokens.


In [67]:
X = tokenizer.texts_to_sequences(train_all['descr_new'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (54214, 256)


In [68]:
Y = pd.get_dummies(train_all['GENRE']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (54214, 27)


In [162]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(43371, 256) (43371, 27)
(10843, 256) (10843, 27)


In [168]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
#model.add(tf.keras.layers.SpatialDropout1D(0.2))
model.add(tf.keras.layers.LSTM(50, return_sequences=True,  dropout=0.25, recurrent_dropout=0.4))
model.add(tf.keras.layers.LSTM(16, return_sequences=False,  dropout=0.25, recurrent_dropout=0.5))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(Y_train.shape[1], activation='softmax'))
opt = tf.keras.optimizers.Adam(learning_rate=5e-3)
loss = tf.keras.losses.CategoricalCrossentropy()
model.compile(loss=loss, optimizer=opt, metrics=['accuracy'])
print(model.summary())

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 256, 100)          2000000   
                                                                 
 lstm_8 (LSTM)               (None, 256, 50)           30200     
                                                                 
 lstm_9 (LSTM)               (None, 16)                4288      
                                                                 
 dense_7 (Dense)             (None, 256)               4352      
                                                                 
 dense_8 (Dense)             (None, 27)                6939      
                                                                 
Total params: 2,045,779
Trainable params: 2,045,779
Non-trainable params: 0
_________________________________________________________________
None


In [169]:
epochs = 5
batch_size = 256

history = model.fit(X_train, Y_train,
                    verbose=1
                    , epochs=epochs, batch_size=batch_size,validation_split=0.3)

Epoch 1/5
119/119 [==============================] - 241s 2s/step - loss: 2.3859 - accuracy: 0.2606 - val_loss: 2.1770 - val_accuracy: 0.4057
Epoch 2/5
119/119 [==============================] - 231s 2s/step - loss: 1.9248 - accuracy: 0.4401 - val_loss: 1.8400 - val_accuracy: 0.4576
Epoch 3/5
119/119 [==============================] - 232s 2s/step - loss: 1.6178 - accuracy: 0.5139 - val_loss: 1.8266 - val_accuracy: 0.4706
Epoch 4/5
119/119 [==============================] - 231s 2s/step - loss: 1.3760 - accuracy: 0.5901 - val_loss: 1.8564 - val_accuracy: 0.4838
Epoch 5/5
119/119 [==============================] - 232s 2s/step - loss: 1.1254 - accuracy: 0.6681 - val_loss: 2.0445 - val_accuracy: 0.4849


In [80]:
train_all.GENRE

0               drama 
1            thriller 
2               adult 
3               drama 
4               drama 
             ...      
54209          comedy 
54210          horror 
54211     documentary 
54212          comedy 
54213         history 
Name: GENRE, Length: 54214, dtype: object

In [88]:
pd.get_dummies(train_all['GENRE']).columns

Index([' action ', ' adult ', ' adventure ', ' animation ', ' biography ',
       ' comedy ', ' crime ', ' documentary ', ' drama ', ' family ',
       ' fantasy ', ' game-show ', ' history ', ' horror ', ' music ',
       ' musical ', ' mystery ', ' news ', ' reality-tv ', ' romance ',
       ' sci-fi ', ' short ', ' sport ', ' talk-show ', ' thriller ', ' war ',
       ' western '],
      dtype='object')

In [89]:
pd.get_dummies(test_solution['GENRE']).columns

Index([' action ', ' adult ', ' adventure ', ' animation ', ' biography ',
       ' comedy ', ' crime ', ' documentary ', ' drama ', ' family ',
       ' fantasy ', ' game-show ', ' history ', ' horror ', ' music ',
       ' musical ', ' mystery ', ' news ', ' reality-tv ', ' romance ',
       ' sci-fi ', ' short ', ' sport ', ' talk-show ', ' thriller ', ' war ',
       ' western '],
      dtype='object')

In [ ]:
#get dummies is the same for train and test datasets. we can use same get_dummies function

In [170]:
# apply token function into test solution dataframe with method .apply()
test_tokenized_desc = test_solution['DESCRIPTION'].apply(tokenize_text)

# добавляем новую колонку в исходный датафрейм
test_solution = test_solution.assign(
    descr_new=test_tokenized_desc
)

test_solution.descr_new.head()

0    [l.r, brane, loves, life, car, apartment, job,...
1    [spain, march, 1964, quico, naughty, child, th...
2    [one, year, life, albin, family, shepherds, no...
3    [father, died, n't, spoken, brother, 10, years...
4    [known, internationally, martial, arts, supers...
Name: descr_new, dtype: object

In [171]:

test_x = tokenizer.texts_to_sequences(test_solution['descr_new'].values)
test_x = pad_sequences(test_x, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', test_x.shape)

Shape of data tensor: (54200, 256)


In [172]:
dicts = pd.get_dummies(train_all['GENRE']).columns
def get_num_dummies(word):
  #num = None
  for i,k in enumerate(dicts):
    if word == k:
      return i
  
  #return [i for i,k in enumerate(dicts) if  str(k) == word]
   
get_num_dummies(' mystery ')

16

In [173]:
test_y = test_solution['GENRE'].apply(get_num_dummies)


In [174]:
pred = model.predict(test_x)

In [146]:
pred.argmax(axis=1)

array([5, 8, 7, ..., 8, 5, 7])

In [147]:
test_y

0        24
1         5
2         7
3         8
4         8
         ..
54195    13
54196    26
54197     1
54198     8
54199     8
Name: GENRE, Length: 54200, dtype: int64

In [175]:
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error, accuracy_score

In [176]:
print('Accuracy score on test dataset is {:.4f}'.format(accuracy_score(test_y, pred.argmax(axis=1))))

Accuracy score on test dataset is 0.4835


In [177]:
import random

In [188]:
for i in range(5):
  
  k = random.randint(0, len(test_x))
  print ('description text: {}'.format(test_solution['DESCRIPTION'][k]))
  print('True genre is: {}'.format(test_solution['GENRE'][k]))
  print('Predict genre is: {}'.format(dicts[pred[k].argmax()]))
  print()

description text:  In 2006, Amanda and Casey Arrington moved from Texas to Durham, North Carolina, where they started a group called Coalition to Unchain Dogs. The purpose of the group, in the long term, was to advocate for the passage of legislation banning or restricting the practice of chaining. In the short term, the group would provide free fences, free spay/neuter, and free basic veterinary care for chained dogs in the community, so that the dogs could have room to run and be free from their chains. This short documentary follows the experience of the Coalition through two of the group's volunteers, Robert and Lori Hensley, as they and the Arringtons rescue Fluffy, a dog severely injured by an embedded tether. Fluffy's story serves as a frame for the Coalition's work and an illustration of the challenges the group has to overcome. The film documents Amanda's skills at building trust with Durham's community members, the deliberations and questions the Arringtons and Hensleys wrest